In [1]:
import ray
import pandas as pd
import time 
import numpy as np

In [ ]:
ray.init(num_cpus=3)

In [ ]:
# Function definitions...

@ray.remote
def example(x):
    time.sleep(np.random.random())
    return np.random.randn()

@ray.remote
class TestCls():
    def __init__(self):
        self.g = 1
        
    def to_go(self, x):
        return x


In [ ]:
# Here, we generate data in redis for remote tasks
results = ray.get([example.remote(x) for x in range(4)])

# Generating data for Actor tasks
actor = TestCls.remote()
actor_results = ray.get([actor.to_go.remote(1)])

In [ ]:
global_state = ray.worker.global_state # Used to access redis client, but also has nice interface for certain information 
gworker = ray.worker.global_worker # Unused for now
rc = global_state.redis_client # Redis Client for interacting with redis


Key things included in redis from `rc.keys()`:
 - Event log
 - worker info
 - Remote functions
 - "Functions to run"
 - Drivers
 - Redis clients
 - Actor classes
 - Actors

In [ ]:
import pprint
pp = pprint.PrettyPrinter() # for printing dicts and lists in a manner easy for the eyes


for k in sorted(rc.keys()):
    try:
        pp.pprint(rc.hgetall(k))
        rc.hgetall(k) 
    except Exception:
        print(k, "Failed ")
# Certain return values, such as `event_log:*`, can only be accessed via list calls to redis

#         try:
#             print('#' * 10)
#             print(k)
#             pp.pprint(rc.lrange(k, 0, -1))
#         except Exception:
#             print(k, "Failed ")
            

In [ ]:
global_state.client_table()

# Tasks

Below is 

In [ ]:
tt = global_state.task_table()
tt_list = list(tt.values())

for d in tt_list:
    d['TaskSpec']['ReturnObjectIDs'] = [oid.hex() for oid in d['TaskSpec']['ReturnObjectIDs']]

In [ ]:
from pandas.io.json import json_normalize

result = json_normalize(tt_list)

In [ ]:
result

We can also parse the event logs in order to get fine grained timing for remote tasks. However, as a user, I'd probably only care about time taken in running the task -- this can be much refined.

In [ ]:
import json 
event_list = []

# Get and decode all task timing/event logs
for key in rc.keys("event_log*"):
    content = rc.lrange(key, 0, -1)
    event_list.append(json.loads(content[0].decode())) 

In [ ]:
from collections import defaultdict

# This seems to be the best way to do the event_log -> dataframe pipeline. 
# First generate a (key, [value]) mapping for all tasks and then apply some pandas operations to convert.

# event_dict is used to store timing info
event_dict = defaultdict(lambda: np.full(len(event_list), np.nan))

# info_dict is used to store meta data - such as function names and task id
info_dict = defaultdict(lambda: [None] * len(event_list))

for i, task_event in enumerate(event_list):
    for event in (task_event):
        time, label, startstop, info = event
        event_dict[(label, startstop)][i] = time
        if info:
            for k in info:
                info_dict[k][i] = info[k]

In [ ]:
# The tuple keys for `event_dict` produce a hierarchical index, which could be useful. However, joining or merging it 
# with other non-hierarchical indices will throw away this structure.

edf = pd.DataFrame(event_dict) 
edf.rename(columns={1: 'start', 2:'end'}, inplace=True)
# edf

In [ ]:
idf = pd.DataFrame(info_dict)
idf.columns = pd.MultiIndex.from_tuples([(c, '') for c in idf]) # this is non-idempotent!
# idf

In [ ]:
remote_table = pd.concat([idf, edf], axis=1)

In [ ]:
# Here, we join two tables together as an example
remote_table.merge(result, left_on="task_id", right_on="TaskSpec.TaskID")

# TODO:

- Error messages logging (specifics - which node, which function call, which actor, what time)
- DataFrame for actor-specific usage - (parent actor, which node, etc.)